In [3]:
#libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import random
import numpy as np
from nltk.corpus import stopwords
import string
from sklearn.base import BaseEstimator, TransformerMixin
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
from gensim.models import Word2Vec
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, RepeatedKFold, GridSearchCV, cross_val_score
from sklearn.model_selection import RepeatedKFold, cross_validate
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import wilcoxon
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from scipy.stats import ttest_rel
from scipy.stats import shapiro
from joblib import dump, load

In [4]:
#classes
class Tokenizer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return [word_tokenize(doc.lower()) for doc in X]


class JoinTokens(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return [' '.join(tokens) for tokens in X]


class StopwordRemover(BaseEstimator, TransformerMixin):
    def __init__(self, stop_words=None):
        self.stop_words = stop_words or set()

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # X è una lista di liste di token
        return [
            [token for token in tokens if token.isalpha() and token not in self.stop_words]
            for tokens in X
        ]

class Word2VecTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vector_size=300, window=5, min_count=2, sg=1):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.sg = sg
        self.model = None

    def fit(self, X, y=None):
        self.model = Word2Vec(
            sentences=X,
            vector_size=self.vector_size,
            window=self.window,
            min_count=self.min_count,
            workers=4,
            sg=self.sg
        )
        return self

    def transform(self, X):
        transformed = []
        for doc in X:
            vectors = [self.model.wv[word] for word in doc if word in self.model.wv]
            if vectors:
                doc_vector = np.mean(vectors, axis=0)
            else:
                doc_vector = np.zeros(self.vector_size)
            transformed.append(doc_vector)
        return np.array(transformed)

In [6]:
import tkinter as tk
from tkinter import messagebox
from joblib import load

# Carica il modello ensemble salvato
model = load('ensemble2.joblib')

# Funzione per fare la predizione
def predict_text():
    user_input = text_entry.get("1.0", tk.END).strip()
    if not user_input:
        messagebox.showwarning("No input inserted", "Insert an abstract for prediction.")
        return
    try:
        prediction = model.predict([user_input])[0]
        messagebox.showinfo("Risult", f"Prediction: Class {prediction}")
    except Exception as e:
        messagebox.showerror("Error", f"Error in prediction:\n{e}")

# Crea l'interfaccia
root = tk.Tk()
root.title("Fake Abstract Detector")

# Campo di inserimento del testo
tk.Label(root, text="Insert a scientific abstract:").pack(pady=5)
text_entry = tk.Text(root, height=10, width=80)
text_entry.pack(padx=10, pady=5)

# Bottone di predizione
predict_button = tk.Button(root, text="Predict", command=predict_text)
predict_button.pack(pady=10)

root.mainloop()
